In [1]:
import os, sys
project_dir = os.path.join(os.getcwd(),'../..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

hyspeclab_dir = os.path.join(project_dir, 'HySpecLab')
if hyspeclab_dir not in sys.path:
    sys.path.append(hyspeclab_dir)

ipdl_dir = os.path.join(project_dir, 'modules/IPDL')
if ipdl_dir not in sys.path:
    sys.path.append(ipdl_dir)    

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir)    

from scipy.io import loadmat
from matplotlib import pyplot as plt
import numpy as np
import config
import torch
from torch import nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
from dataset import DermaDataset

train_dir = ['train', 'validation']
dataset_dir = list(map(lambda x: os.path.join(config.DERMA_DATASET_DIR, x), train_dir))

dataset = DermaDataset(dataset_dir)
X, y = dataset.get(dataframe=False)

In [3]:
from torchvision.transforms import ToTensor

from torch.utils.data import Dataset 
from torchvision import transforms as torchTransforms

class AuxDataset(Dataset):
    def __init__(self, dataset_dir, transform=torchTransforms.Compose([torchTransforms.ToTensor()])):
        super(AuxDataset, self).__init__()
        dataset = DermaDataset(dataset_dir)
        self.X, self.y = dataset.get(dataframe=False)
        self.X, self.y = torch.tensor(self.X, dtype=torch.float), torch.tensor(self.y)
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.X[idx], self.y[idx])

dataset = AuxDataset(dataset_dir, None)
val_size = 512
train_set, val_set = torch.utils.data.random_split(dataset, [len(dataset)-val_size, val_size])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=512, shuffle=False)

In [ ]:
from HySpecLab.feature_selection.nn import KFeatureSelector

# model = nn.Sequential(*[
#     nn.Conv1d(1,1,1, bias=False),
#     nn.Flatten(1),
#     KWinners(116, 50),    
#     nn.Linear(116, 256),
#     nn.ReLU(),
#     nn.Linear(256, 64),
#     nn.ReLU(),
#     nn.Linear(64, 2),
# ])

n_bands = 116
# model = nn.Sequential(*[
#     nn.Conv1d(n_bands,n_bands,1, groups=n_bands, bias=False),
#     nn.Flatten(1),
#     KWinners(n_bands, 25),
#     nn.Linear(n_bands, 256, bias=True),
#     nn.Dropout(0.25),
#     nn.ReLU(),
#     nn.Linear(256, n_bands, bias=True),
#     nn.Sigmoid()
# ])



In [ ]:
n_bands = 116
model = nn.Sequential(*[
    KFeatureSelector(n_bands, 25),
    nn.Linear(n_bands, 64, bias=True),
    nn.Dropout(0.25),
    nn.ReLU(),
    nn.Linear(64, n_bands, bias=True),
    nn.Sigmoid()
])

for m in model.modules():
    if isinstance(m, (nn.Linear, nn.Conv1d)):
        torch.nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
criterion = nn.MSELoss()

from torch.utils.tensorboard import SummaryWriter
from HySpecLab.feature_selection.nn.utils import Reconstruction

tb_writer = SummaryWriter('runs/KFeatureSelector/MSE')
Reconstruction.train(model, optimizer, [train_loader, val_loader], 50, criterion, tb_writer=tb_writer)

In [ ]:
model.eval()
with torch.no_grad():
    input, _ = next(iter(val_loader))
    out = model(input[:,:,None].to(device))
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.plot(input.detach().cpu().numpy()[::64,:].T)
plt.subplot(1,2,2)
plt.plot(out.detach().cpu().numpy()[::64,:].T)
plt.show()

In [ ]:
model.eval()
with torch.no_grad():
    input = input.to(device)
    out = model[0](input[:,:, None])
plt.plot(out[6].detach().cpu().numpy().T)
plt.show()

In [ ]:
a = model[0].model[0].weight.data.cpu().flatten().numpy()
plt.plot(a)
plt.show()

len(a[a>0])

# Test with Cross-Correlation

In [ ]:
from HySpecLab.metrics import NCC

n_bands = 116
model = nn.Sequential(*[
    KFeatureSelector(n_bands, 25),
    nn.Linear(n_bands, 64, bias=True),
    nn.Dropout(0.2),
    nn.ReLU(),
    nn.Linear(64, n_bands, bias=True),
    nn.Sigmoid()
])

for m in model.modules():
    if isinstance(m, (nn.Linear, nn.Conv1d)):
        torch.nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
criterion = NCC()

from torch.utils.tensorboard import SummaryWriter
from HySpecLab.feature_selection.nn.utils import Reconstruction

tb_writer = SummaryWriter('runs/KFeatureSelector/NCC')
Reconstruction.train(model, optimizer, [train_loader, val_loader], 50, criterion, tb_writer=tb_writer)

In [ ]:
model.eval()
with torch.no_grad():
    input, _ = next(iter(val_loader))
    out = model(input[:,:,None].to(device))
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.plot(input.detach().cpu().numpy()[::64,:].T)
plt.subplot(1,2,2)
plt.plot(out.detach().cpu().numpy()[::64,:].T)
plt.show()

In [ ]:
model.eval()
with torch.no_grad():
    input = input.to(device)
    out = model[0](input[:,:, None])
plt.plot(out[6].detach().cpu().numpy().T)
plt.show()

In [ ]:
a = model[0].model[0].weight.data.cpu().flatten().numpy()
plt.plot(a)
plt.show()

In [ ]:
model[0].model[0].weight.data.cpu().flatten()
len(a[a<=0])
len(a[a>0])

# Sparse with KL Div

In [ ]:
from HySpecLab.feature_selection.nn import SparseWeightedFeatureSelector
n_bands = 116
model = nn.Sequential(*[
    SparseWeightedFeatureSelector(n_bands, 1e-5),
    nn.ReLU(),
    nn.Linear(n_bands, 64, bias=True),
    nn.Dropout(0.2),
    nn.ReLU(),
    nn.Linear(64, n_bands, bias=True),
    nn.Sigmoid(),
])

for m in model.modules():
    if isinstance(m, (nn.Linear, nn.Conv1d)):
        torch.nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

from HySpecLab.metrics import NCC
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
criterion = nn.MSELoss()

from torch.utils.tensorboard import SummaryWriter
from HySpecLab.feature_selection.nn.utils import Reconstruction

tb_writer = SummaryWriter('runs/SparseWeightedFeatureSelector/MSE')
Reconstruction.train(model, optimizer, [train_loader, val_loader], 50, criterion, tb_writer=tb_writer)

In [ ]:
weight_data = model[0].weight.data.cpu().numpy()
weight_data_no_negative = weight_data.copy()
weight_data_no_negative[weight_data_no_negative<0] = 0
plt.plot(weight_data_no_negative)
plt.show()

print(len(weight_data[weight_data > 0]))

In [ ]:
model.eval()
with torch.no_grad():
    input, _ = next(iter(val_loader))
    out = model(input[:,:].to(device))
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.plot(input.detach().cpu().numpy()[::64,:].T)
plt.subplot(1,2,2)
plt.plot(out.detach().cpu().numpy()[::64,:].T)
plt.show()